In [1]:
import igraph as ig
import pathlib
import pickle
import tqdm
import time
import pandas as pd
import numpy as np

c:\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [21]:
data_type = 'subsample'
DATA_PATH = f'../data arxiv/hyperedges_list_arxiv_{data_type}.txt'
LINEGRAPH_EDGES_PATH = f'../suplementary/linegraphs/linegraph_edges_list_{data_type}.txt'
LINEGRAPH_NODES_PATH = f'../suplementary/linegraphs/linegraph_nodes_list_{data_type}.txt'
DIST_DIR = "../suplementary/dist"
CLIQUE_PATH = f"../suplementary/clique-projections/{data_type}-proj-graph.txt"


### Data reading

In [6]:
data_path = DATA_PATH
with open(data_path) as file:
    line = file.readline()
hyperedges_list = line.split('], ')
hyperedges_list = [x.replace('[','') for x in hyperedges_list]
hyperedges_list = [x.replace(']','') for x in hyperedges_list] # removes the last "]""
print(len(hyperedges_list))
print(hyperedges_list[0])

506
'math-ph', 'astro-ph.co', 'math.mp'


delete duplicated hyperedges

In [7]:
hyperedges_list = np.unique(hyperedges_list)
print(len(hyperedges_list))
hyperedges_list = [x.replace("'",'') for x in hyperedges_list]
hyperedges_list = [x.split(', ') for x in hyperedges_list]
print(hyperedges_list[0])

506
['astro-ph', 'adap-org', 'nlin.ao', 'physics.plasm-ph']


delete hyperedges of size 1

In [8]:
hyperedges_list = [x for x in hyperedges_list if len(x) > 1]

create dictionaries with hyperedges

In [9]:
def get_nodes_to_edges(edges_to_nodes_dict):
    nodes_to_edges_dict = {}
    for edge, nodes in edges_to_nodes_dict.items():
        for node in nodes:
            if node in nodes_to_edges_dict.keys():
                nodes_to_edges_dict[node].append(edge)
            else:
                nodes_to_edges_dict[node] = [edge]
    return nodes_to_edges_dict

In [10]:
edges_to_nodes_dict = {i: hyperedges_list[i] for i in range(len(hyperedges_list))} # key = hyperedge, value = nodes in hyperedge
nodes_to_edges_dict = get_nodes_to_edges(edges_to_nodes_dict) # key = node, value = list of hyperedges in which the node participates
print(f'nodes count = {len(nodes_to_edges_dict.keys())}')

nodes count = 161


Functions generating weighted linegraph

In [15]:
def weight_function(e1, e2):
    if e1 == e2:
        return (1 / 3) * (len(e1) + 1) - 1
    union_len = len(set(e1).union(e2))
    intersect_len = len(set(e1).intersection(e2))
    return (1 / 3) * union_len * (1 + 1 / intersect_len) - 1


def generate_weighted_line_graph(edges_to_nodes_dict):
    file_path = pathlib.Path(LINEGRAPH_EDGES_PATH)
    
    if not file_path.exists():

        items = list(edges_to_nodes_dict.items())
        line_nodes_ids_dict = {items[i][0]: i for i in range(len(items))}
        line_edges_list = []
        line_edges_weights_list = []

        for i in range(len(items)):
            edge, nodes = items[i]    
            line_edges_list.append([i,i])
            line_edges_weights_list.append(weight_function(nodes, nodes))

        for i in tqdm.tqdm(range(len(items))):    
            edge, nodes = items[i]
            set_nodes = set(nodes)
            neighbors = [k for k,v in items if 
                         not k == edge and
                         (len(v) >= len(nodes)) and 
                         not set_nodes.isdisjoint(v)]
            
            if len(neighbors) > 0:
                for neighbor in neighbors:
                    line_edges_list.append([line_nodes_ids_dict[edge],line_nodes_ids_dict[neighbor]])
                    line_edges_weights_list.append(weight_function(nodes, edges_to_nodes_dict[neighbor]))
        
        G = ig.Graph(n=len(items), edges=line_edges_list, 
                        edge_attrs={'weight': line_edges_weights_list},
                        vertex_attrs={'label': list(line_nodes_ids_dict.keys())})
        edges_list = G.get_edgelist()
        with open(LINEGRAPH_EDGES_PATH,'w') as file:
            lines = []
            for edge_id in range(len(edges_list)):
                edge_weight = G.es[edge_id]["weight"]
                lines.append(f'{edges_list[edge_id][0]} {edges_list[edge_id][1]} {edge_weight}\n')
            file.writelines(lines)
        with open(LINEGRAPH_NODES_PATH ,'w') as file:
            file.writelines([str(G.vs[i]['label'])+'\n' for i in range(len(items))])
    
    with open(LINEGRAPH_EDGES_PATH,'r') as file:
        lines = file.readlines()
    with open(LINEGRAPH_NODES_PATH,'r') as file:
        lines_nodes = file.readlines()
    edges_list = []
    weights = []
    nodes = []
    for line in lines:
        from_node,to_node,weight = line.split(' ')
        from_node = int(from_node)
        to_node = int(to_node)
        if from_node not in nodes:
            nodes.append(from_node)
        if to_node not in nodes:
            nodes.append(to_node)
        weight = float(weight)
        edges_list.append((from_node,to_node))
        weights.append(weight)

    G = ig.Graph(n = len(nodes), edges=edges_list, 
                            edge_attrs={'weight': weights},
                            vertex_attrs={'label': [int(x) for x in lines_nodes]})
    return G



Function calculating hypergraph distanse between nodes u and v

In [16]:
def get_distance(nodes_to_edges_dict, line_graph, u, v, return_path = False):

    from_edges = nodes_to_edges_dict[u]
    to_edges = nodes_to_edges_dict[v]

    if len(from_edges) < len(to_edges):
        u_edges = from_edges
        v_edges = to_edges
    else:
        u_edges = to_edges
        v_edges = from_edges

    line_nodes_ids_dict = {line_graph.vs["label"][i]: i for i in range(len(line_graph.vs["label"]))}

    uv_dist = []
    uv_paths = []
    
    for u_e in u_edges:
        all_paths = line_graph.get_all_shortest_paths(line_nodes_ids_dict[u_e], to = [line_nodes_ids_dict[v_e] for v_e in v_edges], weights=line_graph.es["weight"])
        all_paths_dict = {}
        for i in range(len(all_paths)):
            if len(all_paths[i]) > 1:
                if not all_paths[i][-1] in all_paths_dict.keys():
                    all_paths_dict[all_paths[i][-1]] = [all_paths[i]]
                else:
                    all_paths_dict[all_paths[i][-1]].append(all_paths[i])
        
        for v_e in v_edges:
            if u_e == v_e:
                dist = line_graph.es["weight"][line_graph.get_eid(line_nodes_ids_dict[u_e],line_nodes_ids_dict[v_e])]
                uv_dist.append(dist)
                uv_paths.append([line_graph.get_eid(line_nodes_ids_dict[u_e],line_nodes_ids_dict[v_e])])
            else:
                paths = all_paths_dict[line_nodes_ids_dict[v_e]]
                uv_paths.append(paths)
                if len(paths[0]) > 0:
                    distance = 0
                for i in range (len(paths[0]) - 1):
                    start = paths[0][i]
                    end = paths[0][i + 1]
                    e = line_graph.get_eid(start, end)
                    distance += line_graph.es[e]["weight"]
                    # uv_paths[-1].append((start,end))
                uv_dist.append(distance)
    if not return_path:
        return min(uv_dist) + 1
    i = uv_dist.index(min(uv_dist))
    if len(uv_paths[i]) > 1:
        paths_str = [",".join([str(y) for y in x]) for x in uv_paths[i] ]
        # print(paths_str)
        # print(uv_paths[i])
        paths_str = np.unique(paths_str)
        paths = [x.split(',') for x in paths_str]
        paths = [[int(y) for y in x] for x in paths]
    else:
        paths = uv_paths[i]
    # print(paths)
    return min(uv_dist) + 1, paths

Line graph construction

In [17]:
line_graph = generate_weighted_line_graph(edges_to_nodes_dict)

Distances calculation

In [22]:
dists_dict = {}
paths_dict = {}
node_names = list(nodes_to_edges_dict.keys())
start_id = 0
final_id = len(node_names) - 1
file_path = pathlib.Path(f'{DIST_DIR}\dists_{data_type}-{start_id}-{final_id}.pickle')
    
if not file_path.exists():
    time_start = time.time()
    for id_from in range(start_id,final_id + 1):
        node_from = node_names[id_from]
        dists = []
        paths = []
        print(id_from)
        for node in list(nodes_to_edges_dict.keys()):
            if not node == node_from:
                dist, paths_tmp = get_distance(nodes_to_edges_dict, line_graph, node_from, node, return_path=True)
                dists.append(dist)
                paths.append(paths_tmp)
            else:
                dists.append(0)
                paths.append([])
        dists_dict[node_from] = dists.copy()
        paths_dict[node_from] = paths.copy()
        with open(f'{DIST_DIR}\dists_subsample-{start_id}-{final_id}.pickle', 'wb') as handle:
                pickle.dump((dists_dict, paths_dict), handle)
    time_term = time.time()
    print(f'Elapsed time {time_term - time_start}')
else:
    with open(f'{DIST_DIR}\dists_{data_type}-{start_id}-{final_id}.pickle', 'rb') as handle:
        dists_dict, paths_dict = pickle.load(handle)

Results for hypergraph distances

In [23]:
hyper_dist_df = pd.DataFrame(dists_dict)
hyper_dist_df.index = list(nodes_to_edges_dict.keys())
hyper_dist_df

,astro-ph,adap-org,nlin.ao,physics.plasm-ph,cond-mat,math.oc,nlin.cd,cs.hc,physics.comp-ph,hep-ph,...,q-bio.mn,physics.hist-ph,q-bio.gn,q-bio.cb,q-bio.to,q-fin.rm,q-fin.st,q-fin.tr,funct-an,q-fin.pm
astro-ph,0.000000,1.666667,1.666667,1.666667,1.666667,1.666667,1.333333,1.333333,1.333333,1.333333,...,3.333333,3.333333,5.000000,5.666667,4.000000,4.333333,4.666667,2.500000,4.333333,6.333333
adap-org,1.666667,0.000000,1.666667,1.666667,4.000000,4.666667,4.000000,4.000000,4.000000,4.000000,...,6.333333,6.333333,6.166667,8.500000,7.666667,7.333333,7.666667,5.500000,6.666667,8.333333
nlin.ao,1.666667,1.666667,0.000000,1.666667,1.333333,2.500000,1.333333,2.500000,2.666667,2.500000,...,3.333333,5.444444,2.500000,4.666667,4.000000,4.333333,4.666667,3.500000,3.666667,5.333333
physics.plasm-ph,1.666667,1.666667,1.666667,0.000000,2.666667,3.333333,3.000000,4.000000,2.500000,3.000000,...,5.500000,4.000000,4.333333,6.666667,5.333333,4.666667,4.666667,3.000000,3.666667,5.666667
cond-mat,1.666667,4.000000,1.333333,2.666667,0.000000,1.666667,1.333333,4.000000,2.000000,1.666667,...,4.666667,5.000000,5.000000,5.666667,4.000000,4.333333,4.666667,3.000000,1.666667,6.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
q-fin.rm,4.333333,7.333333,4.333333,4.666667,4.333333,3.000000,2.666667,4.333333,5.166667,5.333333,...,5.666667,6.000000,3.666667,6.666667,4.666667,0.000000,1.666667,5.333333,6.333333,5.000000
q-fin.st,4.666667,7.666667,4.666667,4.666667,4.666667,3.000000,3.000000,4.666667,5.166667,5.333333,...,5.666667,6.000000,3.666667,7.000000,4.666667,1.666667,0.000000,2.000000,6.333333,5.000000
q-fin.tr,2.500000,5.500000,3.500000,3.000000,3.000000,4.000000,3.500000,4.666667,3.500000,3.000000,...,5.666667,6.000000,4.000000,6.333333,6.333333,5.333333,2.000000,0.000000,6.000000,7.000000
funct-an,4.333333,6.666667,3.666667,3.666667,1.666667,3.666667,3.666667,6.000000,3.666667,3.000000,...,6.000000,7.166667,6.000000,7.333333,6.000000,6.333333,6.333333,6.000000,0.000000,7.333333


Function preparing data for cluque projection

In [24]:
def make_clique_projection_data(edges_to_nodes_dict):
    file_path = pathlib.Path(CLIQUE_PATH)
    
    if not file_path.exists():
        proj_edges_dict = {}
        for nodes in edges_to_nodes_dict.values():
            for i in range(len(nodes)-1):
                for j in range(i+1, len(nodes)):
                    node_from = nodes[i]
                    node_to = nodes[j]
                    if not (node_from, node_to) in proj_edges_dict.keys():
                        proj_edges_dict[(node_from,node_to)] = 1
                    else:
                        proj_edges_dict[(node_from,node_to)] += 1
        
        with open(CLIQUE_PATH, 'w') as handle:
            for (node_from, node_to), weight in proj_edges_dict.items():
                handle.write(f'{node_from} {node_to} {weight}')
                handle.write('\n')

    return True

Prepare clique projection graph with igraph and calculate distances in clique projection

In [28]:
make_clique_projection_data(edges_to_nodes_dict)
proj_graph_file = CLIQUE_PATH
with open(proj_graph_file) as file:
        lines = file.readlines()
        edges_list = []
        nodes_dict = {}
        i = -1
        for line in lines:
            from_node, to_node, weight = line.split(" ")
            
            # from_node = int(from_node)
            # to_node = int(to_node)
     
            if not from_node in nodes_dict.keys():
                i += 1
                from_id = i
                nodes_dict[from_node] = i
            else:
                from_id = nodes_dict[from_node]
            if not to_node in nodes_dict.keys():
                i += 1
                to_id = i
                nodes_dict[to_node] = i
            else:
                to_id = nodes_dict[to_node]
            edges_list.append([from_id, to_id])

proj_graph = ig.Graph(edges_list)

proj_dist_dict = {}
proj_dist_path = {}

for from_node in dists_dict.keys():
    from_node_id = nodes_dict[from_node]
    dists = []
    paths_tmp = []
    for node in list(nodes_to_edges_dict.keys()):
        node_id = nodes_dict[node]
        paths = proj_graph.get_shortest_paths(from_node_id, node_id, output = "epath")
        paths_tmp.append(paths[0])
        dists.append(len(paths[0]))

    proj_dist_dict[from_node] = dists
    proj_dist_path[from_node] = paths_tmp

proj_dist_df = pd.DataFrame(proj_dist_dict)
proj_dist_df.index = list(nodes_to_edges_dict.keys())
proj_dist_df

,astro-ph,adap-org,nlin.ao,physics.plasm-ph,cond-mat,math.oc,nlin.cd,cs.hc,physics.comp-ph,hep-ph,...,q-bio.mn,physics.hist-ph,q-bio.gn,q-bio.cb,q-bio.to,q-fin.rm,q-fin.st,q-fin.tr,funct-an,q-fin.pm
astro-ph,0,1,1,1,1,1,1,1,1,1,...,2,2,3,3,2,3,3,2,2,3
adap-org,1,0,1,1,2,2,2,2,2,2,...,3,3,3,3,3,4,4,3,3,4
nlin.ao,1,1,0,1,1,2,1,2,2,2,...,2,2,2,2,2,3,3,2,2,3
physics.plasm-ph,1,1,1,0,2,2,2,2,2,2,...,3,2,3,3,3,3,3,2,2,3
cond-mat,1,2,1,2,0,1,1,2,1,1,...,2,3,2,3,2,3,2,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
q-fin.rm,3,4,3,3,3,2,2,3,3,3,...,3,3,2,4,3,0,1,2,3,2
q-fin.st,3,4,3,3,2,2,2,2,3,3,...,3,3,2,3,3,1,0,1,3,2
q-fin.tr,2,3,2,2,2,2,2,2,2,2,...,3,3,2,3,3,2,1,0,3,3
funct-an,2,3,2,2,1,2,2,3,2,2,...,3,3,3,3,3,3,3,3,0,3


In [31]:
hyper_dist_df.to_csv(f"results_hypergraph_dist_{data_type}.csv")
proj_dist_df.to_csv(f"results_clique-proj_dist_{data_type}.csv")